In [2]:
import evaluate
import datasets
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer

In [3]:
model_id = 'kreimben/CodeMind-gemma'

tokenizer = AutoTokenizer.from_pretrained('kreimben/CodeMind-gemma')
tokenizer.padding_side = 'left'
model = AutoModelForCausalLM.from_pretrained('kreimben/CodeMind-gemma')
model = model.to('cuda')

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
submission_dataset = datasets.load_dataset('csv', data_files='./user_submission_only_python.csv',
                                           split='train').to_pandas()
submission_dataset = submission_dataset[['title_slug', 'question_hints', 'question_content', 'content']]
captions_dataset = datasets.load_dataset('kreimben/leetcode_with_youtube_captions', split='train').to_pandas()[
    ['title_slug', 'question_hints', 'question_content', 'cc_content']]
captions_dataset.rename(columns={'cc_content': 'content'}, inplace=True)

dataset = pd.concat([submission_dataset, captions_dataset])

del submission_dataset, captions_dataset

# dataset.head(3)

In [5]:
from datasets import Dataset

dataset = Dataset.from_pandas(dataset)

In [6]:
GEMMA_2B_IT_MODEL_PREFIX_TEXT = """Below is an coding test problem. Solve the question."""


def generate_prompt(data_point):
    return f"""<start_of_turn>user {GEMMA_2B_IT_MODEL_PREFIX_TEXT}\n
I don't know {data_point['title_slug']} problem. give me the insight or appoach.\n
this is problem's hint.\n{data_point["question_hints"]}\n
here are some content of question.\n{data_point["question_content"]}<end_of_turn>
<start_of_turn>model {data_point["content"]}<end_of_turn>"""

In [7]:
text_column = [generate_prompt(data_point) for data_point in dataset]
dataset = dataset.add_column("prompt", text_column)
dataset = dataset.shuffle(seed=42)  # Shuffle dataset here
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Map:   0%|          | 0/35997 [00:00<?, ? examples/s]

In [8]:
dataset = dataset.remove_columns(['title_slug', 'question_hints', 'question_content', 'content', '__index_level_0__'])

In [9]:
_, test_dataset = dataset.train_test_split(test_size=.1)

In [10]:
# Evaluate the fine-tuned model
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    acc = accuracy.compute(predictions=predictions, references=labels)
    f1_score = f1.compute(predictions=predictions, references=labels, average="weighted")
    return {"accuracy": acc["accuracy"], "f1": f1_score["f1"]}


# Define the trainer
trainer = Trainer(
    model=model,
    # args=training_args,
    tokenizer=tokenizer,
)

C:\Users\aksid\PycharmProjects\CodeMind\venv\lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [11]:
evaluation_metrics = trainer.evaluate(eval_dataset=dataset, metric_key_prefix="test")
print(f"Evaluation metrics: {evaluation_metrics}")

C:\Users\aksid\PycharmProjects\CodeMind\venv\lib\site-packages\transformers\models\gemma\modeling_gemma.py:573: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.02 GiB. GPU 0 has a total capacity of 15.99 GiB of which 0 bytes is free. Of the allocated memory 25.20 GiB is allocated by PyTorch, and 3.08 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)